<a href="https://colab.research.google.com/github/Source-Code777/Machine_Learning_Projects/blob/main/Counselling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GATHERING DATA USING PANDAS WEB_SCRAPPER FROM WBJEE_WEBSITE**

In [ ]:
import pandas as pd
url_23='https://admissions.nic.in/wbjeeb/Applicant/report/orcrreport.aspx?enc=b6w3EPyuw0C4FADZ4v1XmYUz0XFq314fzLjkE3wbM2xr/DbsjpvUS9LBCKXjSeSL'
tables_23=pd.read_html(url_23)
url_24='https://admissions.nic.in/wbjeeb/Applicant/report/orcrreport.aspx?enc=Nm7QwHILXclJQSv2YVS+7l8OpFY/O746kfneOXEneV50mv1B/txHsSKB11hFlsvw'
tables_24=pd.read_html(url_24)

**CREATING DATAFRAMES YEAR-WISE**

In [ ]:
df_23=tables_23[0]
df_24=tables_24[0]

In [ ]:
df_24.sample(5)

In [ ]:
df_23.sample(5)

**RE-NAMING THE FEATURES **

In [ ]:
def preprocess_dataframe(df, year):

    df.columns = df.columns.str.strip()
    rename_map = {
        "Institute": "College_Name",
        "Program": "Branch",
        "Quota": "Domicile",
        "Category": "Reservation"
    }

    df = df.rename(mapper=rename_map,axis=1)
    df["Year"]=year

    return df

 **ADDING YEAR COLUMN IN THE DATAFRAME**

In [ ]:
df_23 = preprocess_dataframe(df_23, 2023)
df_24 = preprocess_dataframe(df_24, 2024)

**CONACATENATING THE DATA-FRAMES**

In [ ]:
df=pd.concat([df_23,df_24],axis=0,ignore_index=False)

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df=df.drop(['Sr.No'],axis=1)

In [ ]:
df.shape
#8093->ROWS,10->COLUMNS

In [ ]:
df.isnull().sum()
#NO NULL VALUE

In [ ]:
df['Reservation'].unique()

In [ ]:
df['Branch'].unique()

In [ ]:
#WE NEED TO REMOVE THE KEYWORD TFW FROM THE VALUES IN BRANCH COLUMN.
# BECAUSE IT IS CREATING DUPLICATE VALUES AND REGARDLESS WE ALREADY HAVE A RESERVATION COLUMN
#ALSO WE NEED TO REPLACE UNWANTED SYMBOLS
df['Branch']=df['Branch'].str.replace("TFW","",regex=False)
df['Branch']=df['Branch'].str.replace("()","",regex=False)
df['Branch']=df['Branch'].str.replace("Tfw","",regex=False)
df['Branch']=df['Branch'].str.replace("-","",regex=False)
df['Branch']=df['Branch'].str.replace(",","",regex=False)

In [ ]:
df["Branch"] = (
    df["Branch"]
    .str.extract(r"\(\s*(.*?)\s*\)")[0]  # extract text inside parentheses
    .fillna(df["Branch"])                # if no parentheses, keep original
    .str.strip()                          # remove leading/trailing spaces
)

# **DATA PREPROCESSING**

In [ ]:
df=df.drop(['Stream'],axis=1)
#Dropping stream because it have a single value

# **CLEANING THE PROGRAM COLUMN**

In [ ]:
df["Branch"].str.replace(r"[^a-zA-Z\s]", "", regex=True)

In [ ]:
df['Branch'].value_counts()

# **CREATING A MAPPING FUNCTION AND APPLYING IT ON BRANCH COLUMN**

In [ ]:
import re
def Cleaning_Func(df,column,min_threshold):

  new_list=df[column]
  class_counts=new_list.value_counts()
  rare_classes = class_counts[class_counts < min_threshold].index
  other_classes=[]

  def map_value(val):
    val_lower = val.lower() # Convert to lowercase for case-insensitive matching
    if val in rare_classes:
      other_classes.append(val)
      return "other"
    elif re.search(r"\bartificial intelligence\b", val_lower):
      return "AI"
    elif re.search(r"\bmachine learning\b", val_lower):
        return "AI"
    elif re.search(r"\bcomputer science\b", val_lower):
      return "CSE"
    elif re.search(r"\biot\b|internet of things\b", val_lower):
      return "IOT"
    elif re.search(r"\bbiotech\b|biotechnology\b", val_lower):
      return "BIO-TECH"
    elif re.search(r"\belectronics\b", val_lower):
      return "ECE"
    elif re.search(r"\bcivil\b", val_lower):
      return "CIVIL"
    elif re.search(r"\bmechanical\b", val_lower):
      return "MECHANICAL"
    elif re.search(r"\bchemical\b", val_lower):
      return "CHEMICAL"
    elif re.search(r"\bproduction\b", val_lower):
      return "PRODUCTION"
    elif re.search(r"\binformation\b",val_lower):
      return "IT"
    elif re.search(r"\belectrical\b",val_lower):
      return "EE"
    else:
      return val

  df[column + "_short"] = new_list.apply(map_value)

  other_classes=list(set(other_classes))

  return df, other_classes

In [ ]:
df, other_list =Cleaning_Func(df, column="Branch",min_threshold=10)
print("Values mapped to 'other':", other_list)

In [ ]:
df.sample(5)

In [ ]:
df=df.drop(["Branch"],axis=1)

In [ ]:
#LET'S DROP THE PHRASE 'Round' from column Round as it is redundant
df['Round']=df['Round'].str.replace("Round","",)

In [ ]:
df.sample(5)

In [ ]:
df.isna().sum()
#NO NULL VALUES

# **VISUALIZING THE DATA**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x="Year", y="Closing Rank", data=df)
plt.ticklabel_format(style='plain', axis='y')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
top_colleges = df.groupby("College_Name")["Closing Rank"].median().sort_values().head(10)
sns.barplot(x=top_colleges.values, y=top_colleges.index)
plt.ticklabel_format(style='plain', axis='x')
plt.title("Top 10 Colleges by Median Closing Rank")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x="Opening Rank", y="Closing Rank", hue="Year", data=df, alpha=0.6)
plt.ticklabel_format(style='plain', axis='both')
plt.title("Opening vs Closing Rank")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
branch_rank = df.groupby("Branch_short")["Closing Rank"].mean().sort_values()
sns.barplot(x=branch_rank.values, y=branch_rank.index)
plt.ticklabel_format(style='plain', axis='x')
plt.title("Average Closing Rank by Branch")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x="Year", hue="Branch_short", data=df, palette="Set2")
plt.title("Branch_Demand_Year_wise")
plt.legend(bbox_to_anchor=(1,1.05), loc='upper left')
plt.show()

# **LET'S DETECT OUTLIERS **

In [ ]:
import numpy as np

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower) | (df[column] > upper)]
    return outliers, lower, upper

outliers_open, low_open, up_open = detect_outliers(df, "Opening Rank")
outliers_close, low_close, up_close = detect_outliers(df, "Closing Rank")

print("Opening Rank outliers:", len(outliers_open))
print("Closing Rank outliers:", len(outliers_close))

# **CLEANING THE VALUES IN THE COLUMN AND CLASSIFYING THE COLLEGE'S AS GOVERMENT AND PRIVATE**

In [ ]:
import pandas as pd
import re

df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

df["college_name"] = (
    df["college_name"]
    .str.replace("Govt.", "Government", regex=False)
    .str.replace("Goverment", "Government", regex=False)
    .str.replace("&", "and", regex=False)
    .str.replace(r"\.\.\.$", "", regex=True)
    .str.title()
)

df["opening_rank"] = pd.to_numeric(df["opening_rank"], errors="coerce")
df["closing_rank"] = pd.to_numeric(df["closing_rank"], errors="coerce")
df["year"] = pd.to_numeric(df["year"], errors="coerce")

df["seat_type"] = df["seat_type"].str.replace(" Seats", "", regex=False)
df["quota"] = df["quota"].replace({"Home State": "Home", "All India": "AI"})
df["category"] = df["category"].replace({"Tuition Fee Waiver": "TFW"})

gov_keywords = ["government", "govt", "university of calcutta", "calcutta university",
                "jadavpur university", "presidency university", "makaut", "wbut",
                "kalyani university", "burdwan university", "vidyasagar university",
                "north bengal university", "west bengal state university", "aliah university"]

def classify(name):
    s = str(name).lower()
    return "Government" if any(g in s for g in gov_keywords) else "Private"

df["college_type"] = df["college_name"].apply(classify)
df["rank_range"] = df["closing_rank"] - df["opening_rank"]

df["program_group"] = df["program_ref"].str.lower().map(
    lambda x: "CSE" if "cse" in x else ("ECE" if "ece" in x else "OTHER")
)

print("df cleaned and classified")

CLASSIFYING COLLEGES INTO HARD MEDIUM AND EASY BASED ON CLOSING RANK AND DROPPING THE RANGE COLUMN AS IT WAS MISLEADING

In [ ]:
df["difficulty"] = pd.qcut(df["closing_rank"], 3, ["Hard", "Medium", "Easy"])
df=df.drop(['rank_range'],axis=1)
df["seat_category"] = df["seat_type"] + "_" + df["category"]

In [ ]:
df=df.drop(["seat_type","category"],axis=1)

In [ ]:
df.sample(5)

In [ ]:
# Pivot table: counts
pivot_counts = pd.crosstab(df["seat_category"], df["difficulty"])

# Pivot table: percentages (row-wise)
pivot_percent = pd.crosstab(df["seat_category"], df["difficulty"], normalize="index") * 100

print("=== Counts ===")
print(pivot_counts)
print("\n=== Percentages ===")
print(pivot_percent.round(2))

In [ ]:
import matplotlib.pyplot as plt

# Pivot table: counts
pivot_counts = pd.crosstab(df["seat_category"], df["difficulty"])

# Pivot table: percentages
pivot_percent = pd.crosstab(df["seat_category"], df["difficulty"], normalize="index") * 100

# Plot 1: Counts
pivot_counts.plot(kind="bar", stacked=True, figsize=(10,6))
plt.title("Difficulty Distribution by Seat Category (Counts)")
plt.xlabel("Seat Category")
plt.ylabel("Number of Programs")
plt.xticks(rotation=45)
plt.legend(title="Difficulty")
plt.show()

In [ ]:
pivot_percent.plot(kind="bar", stacked=True, figsize=(10,6), colormap="tab20c")
plt.title("Difficulty Distribution by Seat Category (Percentages)")
plt.xlabel("Seat Category")
plt.ylabel("Percentage (%)")
plt.xticks(rotation=45)
plt.legend(title="Difficulty")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

pivot_percent_ct = pd.crosstab(df["college_type"], df["difficulty"], normalize="index") * 100

pivot_percent_ct.plot(kind="bar", stacked=True, figsize=(8,5), colormap="tab20c")
plt.title("Difficulty Distribution by College Type (Percentages)")
plt.xlabel("College Type")
plt.ylabel("Percentage (%)")
plt.xticks(rotation=0)
plt.legend(title="Difficulty")
plt.show()

In [ ]:
df.sample(5)

In [ ]:
agg_scores = (
    df.groupby(["college_name", "program_ref"])["closing_rank"]
    .mean()
    .reset_index()
    .rename(columns={"closing_rank": "avg_closing_rank"})
)

df = pd.merge(df, agg_scores, on=["college_name", "program_ref"], how="left")

df["competitiveness_score"] = 1 / (df["avg_closing_rank"] + 1)

In [ ]:
df=df.drop(['avg_closing_rank'],axis=1)

In [ ]:
df.sample(5)

In [ ]:
df['college_program'] = df['college_name'] + ' - ' + df['program_ref']


In [ ]:
df_class=df.drop(['college_name','program_ref'],axis=1)

In [ ]:
df_class.sample(5)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# =========================
# 1. Define features
# =========================
numerical_features = ['opening_rank', 'closing_rank', 'competitiveness_score']
categorical_features = ['quota', 'seat_category', 'college_type', 'program_group', 'difficulty']
ordinal_features = ['round']

# =========================
# 2. Preprocessor
# =========================
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('ord', OrdinalEncoder(categories=[[1,2,3]], handle_unknown='use_encoded_value', unknown_value=-1), ordinal_features)
    ],
    remainder='drop'
)

# =========================
# 3. Target variable encoding
# =========================
label_encoder = LabelEncoder()
y = df['college_program']
y_encoded = label_encoder.fit_transform(y)

# =========================
# 4. Preprocess X
# =========================
X = df.drop(columns=['college_program'])
X_preprocessed = preprocessor.fit_transform(X)

# =========================
# 5. Apply PCA
# =========================
# 2D PCA
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_preprocessed)

# 3D PCA
pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_preprocessed)

# =========================
# 6. Explained variance
# =========================
print("Explained variance ratio (2D):", pca_2d.explained_variance_ratio_)
print("Explained variance ratio (3D):", pca_3d.explained_variance_ratio_)

# =========================
# 7. 2D Visualization
# =========================
plt.figure(figsize=(12,8))
scatter = plt.scatter(X_pca_2d[:,0], X_pca_2d[:,1], c=y_encoded, cmap='tab20', alpha=0.7)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA 2D Visualization of College Data')
plt.colorbar(scatter, label='Encoded College Program')
plt.show()

# =========================
# 8. 3D Visualization
# =========================
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X_pca_3d[:,0], X_pca_3d[:,1], X_pca_3d[:,2], c=y_encoded, cmap='tab20', alpha=0.7)
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
ax.set_title('PCA 3D Visualization of College Data')
fig.colorbar(p, label='Encoded College Program')
plt.show()
